In [ ]:
from whisper import whisper
from pytube import YouTube
import pandas as pd


In [ ]:
# Get video transcription
model = whisper.load_model('base')
v_url = "https://www.youtube.com/watch?v=Rm8iK_FlPCk"
youtube_video = YouTube(v_url) # Select YT video
audio_stream_set = youtube_video.streams.filter(only_audio = True)
audio_stream = audio_stream_set.first() # Select quality audio stream
audio_stream.download(filename = 'test_video.mp4') # Download video
output = model.transcribe("/content/text_video.mp4") # Get transcript
output

In [ ]:
# Split transcript and save into CSV file
import pandas as pd
import nltk

nltk.download('punkt')  # download the NLTK tokenizer
transcript = output['text']
# create a Pandas DataFrame with one row for each sentence
trans_df = pd.DataFrame({'sentence': nltk.sent_tokenize(transcript)})
# add a new column with the length of each sentence
trans_df['sentence_length'] = trans_df['sentence'].apply(len)
# print the DataFrame
print(trans_df)
# save the DataFrame to a CSV file
trans_df.to_csv('text_video.csv', index=False)
print("saved")



In [ ]:
df = pd.read_csv('/content/test_video.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(10)